In [1]:

# libraries
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import gc
from matplotlib import style
from pybaseball import statcast
import progressbar 
import warnings
import psycopg2

# pd.options.mode.chained_assignment = None  # default='warn'
# warnings.simplefilter(action='ignore', category=FutureWarning)
# pd.options.display.float_format = '{:.3f}'.format

df_sc = statcast(start_dt='2024-01-01', end_dt='2024-09-30')



This is a large query, it may take a moment to complete
Skipping offseason dates


  8%|▊         | 17/200 [00:14<02:04,  1.47it/s]/opt/homebrew/anaconda3/lib/python3.11/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):
/opt/homebrew/anaconda3/lib/python3.11/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):
  9%|▉         | 18/200 [00:15<02:00,  1.51it/s]/opt/homebrew/anaconda3/lib/python3.11/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):
 10%|▉         | 19/200 [00:15<01:47,  1.68it/s]/opt/homebrew/anaconda3/lib/python3.11/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is depr

In [ ]:
df_sc.pitc



,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,n_thruorder_pitcher,n_priorpa_thisgame_player_at_bat,pitcher_days_since_prev_game,batter_days_since_prev_game,pitcher_days_until_next_game,batter_days_until_next_game,api_break_z_with_gravity,api_break_x_arm,api_break_x_batter_in,arm_angle
202,FF,2024-09-30,97.4,-2.1,4.88,"Díaz, Edwin",518595,621242,field_out,hit_into_play,...,1,4,1,1,3,1,1.4,0.96,0.96,17.6
205,SL,2024-09-30,90.7,-2.14,5.06,"Díaz, Edwin",518595,621242,NaN,ball,...,1,4,1,1,3,1,2.14,-0.2,-0.2,23.1
209,SL,2024-09-30,91.1,-2.07,5.14,"Díaz, Edwin",518595,621242,NaN,swinging_strike,...,1,4,1,1,3,1,2.37,-0.12,-0.12,22.4
214,SL,2024-09-30,91.3,-2.05,5.07,"Díaz, Edwin",518595,621242,NaN,ball,...,1,4,1,1,3,1,2.09,-0.21,-0.21,22.4
221,SL,2024-09-30,89.1,-2.13,5.15,"Díaz, Edwin",518595,621242,NaN,swinging_strike,...,1,4,1,1,3,1,2.2,-0.17,-0.17,20.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1721,None,2024-03-15,<NA>,<NA>,<NA>,"Singer, Brady",666310,663903,None,swinging_strike,...,1,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1776,None,2024-03-15,<NA>,<NA>,<NA>,"Singer, Brady",666310,663903,None,swinging_strike,...,1,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1859,None,2024-03-15,<NA>,<NA>,<NA>,"Singer, Brady",608070,663903,double,hit_into_play,...,1,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1953,None,2024-03-15,<NA>,<NA>,<NA>,"Singer, Brady",665926,663903,field_out,hit_into_play,...,1,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [10]:



import pandas as pd

# Mapping
result_map = {
    'pitchout_hit_into_play_score': 'contact',
    'hit_into_play_score': 'contact',
    'hit_into_play': 'contact',
    'hit_by_pitch': 'contact',
    'hit_into_play_no_out': 'contact',
    'pitchout_hit_into_play_no_out': 'contact',
    'pitchout_hit_into_play': 'contact',
    'foul_pitchout': 'foul',
    'foul_bunt': 'foul',
    'bunt_foul_tip': 'foul',
    'foul': 'foul',
    'foul_tip': 'foul',
    'intent_ball': 'take',
    'blocked_ball': 'take',
    'pitchout': 'take',
    'ball': 'take',
    'called_strike': 'take',
    'unknown_strike': 'whiff',
    'swinging_strike_blocked': 'whiff',
    'swinging_pitchout': 'whiff',
    'missed_bunt': 'whiff',
    'swinging_strike': 'whiff'
}

# Apply mapping
df_sc['result_type'] = df_sc['description'].map(result_map)

# Define what counts as a "swing"
# df_sc.groupby(['batter', 'pitch_type'])['result_type'].value_counts(normalize=True).unstack().fillna(0).reset_index()

df_swing_pct = df_sc.groupby(['batter', 'pitch_type', 'zone']).agg(
    contact_pct=('result_type', lambda x: (x == 'contact').mean()),
    foul_pct=('result_type', lambda x: (x == 'foul').mean()),
    whiff_pct=('result_type', lambda x: (x == 'whiff').mean()),
    take_pct=('result_type', lambda x: (x == 'take').mean()),
    count=('result_type', 'count')
).reset_index()


df_swing_pct











,batter,pitch_type,zone,contact_pct,foul_pct,whiff_pct,take_pct,count
0,444482,CH,1,0.500000,0.166667,0.000000,0.333333,6
1,444482,CH,4,1.000000,0.000000,0.000000,0.000000,8
2,444482,CH,5,0.333333,0.333333,0.333333,0.000000,3
3,444482,CH,6,0.750000,0.250000,0.000000,0.000000,4
4,444482,CH,7,0.363636,0.272727,0.363636,0.000000,11
...,...,...,...,...,...,...,...,...
65838,815888,FC,9,0.000000,1.000000,0.000000,0.000000,1
65839,815888,FC,12,0.000000,0.000000,0.000000,1.000000,1
65840,815888,FF,4,0.000000,0.000000,1.000000,0.000000,1
65841,815888,FF,11,0.500000,0.500000,0.000000,0.000000,2


In [ ]:
import psycopg2
import pandas as pd

# Connect to PostgreSQL
conn = psycopg2.connect(
    dbname="postgres",
    user="postgres",
    password="Gabriela3$",
    host="piedmont-db.cp6uayweg6ql.us-east-2.rds.amazonaws.com",
    port="5432"
)
cur = conn.cursor()

# Ensure the table exists
cur.execute("""
    CREATE TABLE IF NOT EXISTS pitcher_pitch_covariance_mean (
        pitcher BIGINT,
        player_name TEXT,
        pitch_type TEXT,
        mean_plate_x FLOAT,
        mean_plate_z FLOAT,
        cov_plate_xx FLOAT,
        cov_plate_zz FLOAT,
        cov_plate_xz FLOAT
    )
""")

# Insert data
for _, row in pitcher_covariance_mean.iterrows():
    cur.execute("""
        INSERT INTO pitcher_pitch_covariance_mean (pitcher, player_name, pitch_type, mean_plate_x, mean_plate_z, cov_plate_xx, cov_plate_zz, cov_plate_xz)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
    """, (row['pitcher'], row['player_name'], row['pitch_type'], row['mean_plate_x'], row['mean_plate_z'], row['cov_plate_xx'], row['cov_plate_zz'], row['cov_plate_xz']))

# Commit and close connection
conn.commit()
cur.close()
conn.close()

print("Data successfully inserted into PostgreSQL!")
